In [1]:
import pandas as pd
import numpy as np
import replicaEVSE.load_curve as sim
import os
import joblib
import dask.dataframe as dd

%reload_ext autoreload
%autoreload 2



pd.set_option('display.max_columns', None)

datadir = '../../data'
mode = 'ON_DEMAND_AUTO'
simulation_id = 'dev'
test = True

#Created in the EIA_data_download.ipynb notebook
existing_load=pd.read_csv('../../data/EIA_demand_summary.csv') 
if test:
    # df = pd.read_parquet(os.path.join(datadir, 'wa_pop_and_trips_subsample.parquet'))
    df = dd.read_parquet(os.path.join(datadir, 'wa_pop_and_trips_sorted.parquet'))
    # df = df.loc[df['mode'] == mode]
    df = df.sample(frac=0.001, random_state=1)
    # df = df.sort_values(by=['person_id', 'start_time']).reset_index(drop=True)
    simulation_id = 'dev'

else: 


    # read in the joined trips and population data sets
    merged_df = pd.read_parquet(os.path.join(datadir, 'wa_pop_and_trips_sorted.parquet'))

    # right now, only look at private auto trips
    df = merged_df.loc[merged_df['mode'] == mode]



### get only the tnc trips

In [ ]:
df = df.loc[df['mode'] == 'ON_DEMAND_AUTO']
df = df.sort_values(by=['person_id', 'start_time']).reset_index(drop=True)
df = df.compute()

Two ways to do this: 
 1. everyone charges when they are ~empty only. in this case they would take as many trips as they need and charge at either a L2 or DCFC charger for as long as they need to get back to full. 
 Obviously not realistic. 

 2. Charge after x trips and then charge as long as needed to get back the energy lost in those x trips as long as dont exceed some threshold distance then stop and charge. Charging is drawn from a distribution of L2 and DCFC. 

In [ ]:
max_trip_distance = 100
max_trips = 5
tot_distance = 0
trip_count = 0
kwh_per_mi = 0.3
which_charge_rate = sim.determine_charger_availability_tnc()
charge = 0
for row in len(df):
    trip = df.iloc[row,]
    tot_distance += trip['distance_miles']
    if tot_distance > max_trip_distance:
        # exceeded the distance limit
        # start a new trip
        trip_count = 0
        tot_distance = 0

    if trip_count > max_trips:
        # we should charge
        charge += tot_distance * kwh_per_mi # mi * kwh/mi 
        # add to load here
        # calc time to load up to max distance
        #  determine_charger_availability_tnc in kw/h
    
    # charge duration = charge_used / charge rate
    charge_duration = charge * which_charge_rate

    # this needs to get saved somewere. 
    charge_time = trip['end_time'] + pd.Timedelta(f"{charge_duration} hour")




In [ ]:
# this is the instance where we see how many trips it took to get to 
tot_distance = 0
for row in len(df)
    trip = df.iloc[row,]
    tot_distance += trip['distance_miles']
    if tot_distance > 100:
        tot_distance = 0

# What about a fraction of the private_auto? 

Could find the ones with number of trips equal to some number from another source

In [27]:
df = pd.read_parquet(os.path.join(datadir, 'wa_pop_and_trips_sorted.parquet'))
df = df.loc[(df['mode'] == 'PRIVATE_AUTO') & (df['weekday'] == 'thursday')]

In [28]:
df['person_id'].value_counts()

Dask Series Structure:
npartitions=1
    int64
      ...
Name: count, dtype: int64
Dask Name: value-counts-agg, 10 graph layers

In [26]:
col = 'person_id'

In [31]:
counts = df[col].value_counts()
valids = counts[counts>20].index

ubers = df[df[col].isin(valids)]
vmt = ubers.groupby(col)['distance_miles'].sum()

In [34]:
ubers.travel_purpose.value_counts().compute()

KeyboardInterrupt: 

In [22]:
vmt.describe()

count    11593.000000
mean       184.050749
std        139.656769
min          6.808621
25%         94.172059
50%        146.594494
75%        228.365180
max       1614.991545
Name: distance_miles, dtype: float64

In [23]:
vmt.min()

6.808621473360765